In [1]:
#import dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pymongo

# SQL Alchemy
from sqlalchemy import create_engine

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

# Config variables
from config import local_db_user, local_db_pwd, local_db_endpoint, local_db_port, local_db_name

In [2]:
# Create Engine and Pass in MySQL Connection

# local Postgres DB Connection  
local_engine = create_engine(f"postgresql://{local_db_user}:{local_db_pwd}@{local_db_endpoint}:{local_db_port}")
#print(f"postgresql://{local_db_user}:{local_db_pwd}@{local_db_endpoint}:{local_db_port}")

# Create a local database engine connection
local_conn = local_engine.connect()

# Create local database
try:
    local_conn.execution_options(isolation_level="AUTOCOMMIT").execute(f"CREATE DATABASE {local_db_name}")
except Exception as e:
    print(e)

# Connect to local database
try:
    local_engine = create_engine(f"postgresql://{local_db_user}:{local_db_pwd}@{local_db_endpoint}:{local_db_port}/{local_db_name}")
    local_conn = local_engine.connect()
except Exception as e:
    print(e)  
    
#confirm tables
local_engine.table_names()

(psycopg2.errors.DuplicateDatabase) database "project2_db" already exists

[SQL: CREATE DATABASE project2_db]
(Background on this error at: http://sqlalche.me/e/13/f405)


['2020_fantasy_stats']

In [3]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\Steven\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [4]:
# URL of page to be scraped

#for season in range(2010, 20210):
season = '2020'
url = (f'https://fantasydata.com/nfl/fantasy-football-leaders?season={season}&seasontype=1&scope=1&subscope=1&startweek=1&endweek=1&aggregatescope=1&range=1')
url

# Retrieve page with the browser module
browser.visit(url)

# setup html parser
html = browser.html
fantasy_soup = BeautifulSoup(html, 'html.parser')

In [5]:
# load 300 rows view
load_300 = browser.links.find_by_partial_text('300')
load_300.click()

# load more rows
load_more = browser.links.find_by_partial_text('Load More')
load_more[1].click()
# for x in range(6):
#     load_more[1].click()

ElementClickInterceptedException: Message: element click intercepted: Element <a ng-hide="LoadingMore" ng-click="LoadMore()">...</a> is not clickable at point (674, 614). Other element would receive the click: <iframe id="google_ads_iframe_/20842576/GNG2YM/GNG2YM-DDA.B_0" title="3rd party ad content" name="google_ads_iframe_/20842576/GNG2YM/GNG2YM-DDA.B_0" width="728" height="90" scrolling="no" marginwidth="0" marginheight="0" frameborder="0" sandbox="allow-forms allow-popups allow-popups-to-escape-sandbox allow-same-origin allow-scripts allow-top-navigation-by-user-activation" style="border: 0px; vertical-align: bottom;" data-google-container-id="e" data-load-complete="true"></iframe>
  (Session info: chrome=88.0.4324.104)


In [6]:
# save the stats table as a df
stats_table = fantasy_soup.find("table", id="stats_grid")
stats_df = pd.read_html(f'<table>${stats_table.tbody}</table>')[0]
stats_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,BUF,QB,16,4544,37,10,421,8,1,12,1,0,0,0,0,24.7,395.06
1,GB,QB,16,4299,48,5,149,3,1,-6,0,0,0,0,0,23.9,382.26
2,ARI,QB,16,3971,26,12,819,11,0,0,0,0,0,0,0,23.7,378.74
3,KC,QB,15,4740,38,6,308,2,0,0,0,0,0,0,0,25.0,374.40
4,HOU,QB,16,4823,33,7,444,3,0,0,0,0,0,0,0,23.1,369.32
5,SEA,QB,16,4212,40,13,513,2,0,0,0,0,0,0,0,22.5,359.78
6,TEN,QB,16,3819,33,7,266,7,1,0,0,0,0,0,0,21.5,343.36
7,TB,QB,16,4633,40,12,6,3,0,0,0,0,0,0,0,21.1,337.92
8,LAC,QB,15,4336,31,10,234,5,0,0,0,0,0,0,0,22.2,332.84
9,BAL,QB,15,2757,26,9,1005,7,0,0,0,0,0,0,0,22.2,332.78


In [7]:
# save the names table as a df
table_names = fantasy_soup.find("div", class_="k-grid-content-locked")
names_df = pd.read_html(f'<table>${table_names.tbody}</table>')[0]
names_df

,0,1
0,1,Josh Allen
1,2,Aaron Rodgers
2,3,Kyler Murray
3,4,Patrick Mahomes
4,5,Deshaun Watson
5,6,Russell Wilson
6,7,Ryan Tannehill
7,8,Tom Brady
8,9,Justin Herbert
9,10,Lamar Jackson


In [8]:
col_names = ['RK', 'NAME', 'TEAM', 'POS', 'GMS', 'PASS YDS', 'PASS TD', 'PASS INT', 'RUSH YDS', 'RUSH TD', 'REC', 'REC YDS', 'REC TD', 'DEF SCK', 'DEF INT', 'DEF FF', 'DEF FR', 'FPTS/G', 'FPTS']
combined_df = pd.concat([names_df, stats_df], axis=1)
combined_df.columns = [col_names]
combined_df

,RK,NAME,TEAM,POS,GMS,PASS YDS,PASS TD,PASS INT,RUSH YDS,RUSH TD,REC,REC YDS,REC TD,DEF SCK,DEF INT,DEF FF,DEF FR,FPTS/G,FPTS
0,1,Josh Allen,BUF,QB,16,4544,37,10,421,8,1,12,1,0,0,0,0,24.7,395.06
1,2,Aaron Rodgers,GB,QB,16,4299,48,5,149,3,1,-6,0,0,0,0,0,23.9,382.26
2,3,Kyler Murray,ARI,QB,16,3971,26,12,819,11,0,0,0,0,0,0,0,23.7,378.74
3,4,Patrick Mahomes,KC,QB,15,4740,38,6,308,2,0,0,0,0,0,0,0,25.0,374.40
4,5,Deshaun Watson,HOU,QB,16,4823,33,7,444,3,0,0,0,0,0,0,0,23.1,369.32
5,6,Russell Wilson,SEA,QB,16,4212,40,13,513,2,0,0,0,0,0,0,0,22.5,359.78
6,7,Ryan Tannehill,TEN,QB,16,3819,33,7,266,7,1,0,0,0,0,0,0,21.5,343.36
7,8,Tom Brady,TB,QB,16,4633,40,12,6,3,0,0,0,0,0,0,0,21.1,337.92
8,9,Justin Herbert,LAC,QB,15,4336,31,10,234,5,0,0,0,0,0,0,0,22.2,332.84
9,10,Lamar Jackson,BAL,QB,15,2757,26,9,1005,7,0,0,0,0,0,0,0,22.2,332.78


In [19]:
combined_df
#combined_df.dtypes

,RK,NAME,TEAM,POS,GMS,PASS YDS,PASS TD,PASS INT,RUSH YDS,RUSH TD,REC,REC YDS,REC TD,DEF SCK,DEF INT,DEF FF,DEF FR,FPTS/G,FPTS
0,1,Josh Allen,BUF,QB,16,4544,37,10,421,8,1,12,1,0,0,0,0,24.7,395.06
1,2,Aaron Rodgers,GB,QB,16,4299,48,5,149,3,1,-6,0,0,0,0,0,23.9,382.26
2,3,Kyler Murray,ARI,QB,16,3971,26,12,819,11,0,0,0,0,0,0,0,23.7,378.74
3,4,Patrick Mahomes,KC,QB,15,4740,38,6,308,2,0,0,0,0,0,0,0,25.0,374.40
4,5,Deshaun Watson,HOU,QB,16,4823,33,7,444,3,0,0,0,0,0,0,0,23.1,369.32
5,6,Russell Wilson,SEA,QB,16,4212,40,13,513,2,0,0,0,0,0,0,0,22.5,359.78
6,7,Ryan Tannehill,TEN,QB,16,3819,33,7,266,7,1,0,0,0,0,0,0,21.5,343.36
7,8,Tom Brady,TB,QB,16,4633,40,12,6,3,0,0,0,0,0,0,0,21.1,337.92
8,9,Justin Herbert,LAC,QB,15,4336,31,10,234,5,0,0,0,0,0,0,0,22.2,332.84
9,10,Lamar Jackson,BAL,QB,15,2757,26,9,1005,7,0,0,0,0,0,0,0,22.2,332.78


In [9]:
#upload dataframe to database

combined_df.to_sql(name=f'2020_fantasy_stats', if_exists='replace', con=local_conn, index=False)
    

KeyError: "('RK',"